In [3]:
from google.cloud import bigquery
import csv
import numpy as np
import pandas as pd
#import hive_connector
import os

credentials_path = 'data-key.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path


client = bigquery.Client()

QUERY = ('SELECT YearsExperience,Salary FROM `sodium-reporter-331815.mldata1.input_data` LIMIT 1000')
query_job = client.query(QUERY) # API request
#query_job.to_csv("id.CSV",index= False)
rows = query_job.result()  # Waits for query to finish

#Hive
#rows = pd.read_sql("SELECT YearsExperience,Salary FROM `hip-plexus-238820.mldata1.input_data` LIMIT 1000", conn)
header = ['YearsExperience', 'Salary']

array =[]
for row in rows:
    a =[]
    a.append(row.YearsExperience)
    a.append(row.Salary)
    array.append(a)
    
print(array)

[[1.1, 39343.0], [1.3, 46205.0], [1.5, 37731.0], [2.0, 43525.0], [2.2, 39891.0], [2.9, 56642.0], [3.0, 60150.0], [3.2, 54445.0], [3.2, 64445.0], [3.7, 57189.0], [3.9, 63218.0], [4.0, 55794.0], [4.0, 56957.0], [4.1, 57081.0], [4.5, 61111.0], [4.9, 67938.0], [5.1, 66029.0], [5.3, 83088.0], [5.9, 81363.0], [6.0, 93940.0], [6.8, 91738.0], [7.1, 98273.0], [7.9, 101302.0], [8.2, 113812.0], [8.7, 109431.0], [9.0, 105582.0], [9.5, 116969.0], [9.6, 112635.0], [10.3, 122391.0], [10.5, 121872.0]]


In [4]:
with open('test.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(array)
    
    
dataset = pd.read_csv('test.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 1].values

print(dataset)

# Importing the dataset
dataset = pd.read_csv('Salary_Data.csv')
X_train = dataset.iloc[:, :-1].values
y_train = dataset.iloc[:, 1].values

#print(dataset)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0)

dataset = pd.read_csv('test.csv')
X_test = dataset.iloc[:, :-1].values
y_test = dataset.iloc[:, 1].values


print(X_test)



# Feature Scaling
"""from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train)"""

# Fitting Simple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)
for value in y_pred:
    print(value)


#Upload data to bigquery

table_id = 'sodium-reporter-331815.mldata1.prediction_data'

for value in y_pred:
    rows_to_insert = [
     {u'salary':value},
     ]
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print('New rows have been added.')
    else:
        print(f'Encountered errors while inserting rows: {errors}')

    YearsExperience    Salary
0               1.1   39343.0
1               1.3   46205.0
2               1.5   37731.0
3               2.0   43525.0
4               2.2   39891.0
5               2.9   56642.0
6               3.0   60150.0
7               3.2   54445.0
8               3.2   64445.0
9               3.7   57189.0
10              3.9   63218.0
11              4.0   55794.0
12              4.0   56957.0
13              4.1   57081.0
14              4.5   61111.0
15              4.9   67938.0
16              5.1   66029.0
17              5.3   83088.0
18              5.9   81363.0
19              6.0   93940.0
20              6.8   91738.0
21              7.1   98273.0
22              7.9  101302.0
23              8.2  113812.0
24              8.7  109431.0
25              9.0  105582.0
26              9.5  116969.0
27              9.6  112635.0
28             10.3  122391.0
29             10.5  121872.0
[[ 1.1]
 [ 1.3]
 [ 1.5]
 [ 2. ]
 [ 2.2]
 [ 2.9]
 [ 3. ]
 [ 3.2]
 [ 3.2